In [1]:
import torch
import torch.nn.functional as F
from math import pi
import cv2 as cv
import kornia as K

In [3]:


def sample_homography(
        shape, perspective=True, scaling=True, rotation=True, translation=True,
        n_scales=5, n_angles=25, scaling_amplitude=0.1, perspective_amplitude_x=0.1,
        perspective_amplitude_y=0.1, patch_ratio=0.5, max_angle=pi/2,
        allow_artifacts=False, translation_overflow=0.):
    """采样随机有效单应性变换。

    计算原始图像中的随机块与相同图像大小的变形投影之间的单应性变换。
    与kornia.geometry.transform一样，它将输出（变形块）映射到变���后的输入点（原始块）。
    块（初始化为简单的半尺寸中心裁剪）被迭代地投影、缩放、旋转和平移。

    参数:
        shape: 指定原始图像高度和宽度的二维张量。
        perspective: 布尔值，启用透视和仿射变换。
        scaling: 布尔值，启用块的随机缩放。
        rotation: 布尔值，启用块的随机旋转。
        translation: 布尔值，启用块的随机平移。
        n_scales: 缩放时采样的尝试尺度数量。
        n_angles: 旋转时采样的尝试角度数量。
        scaling_amplitude: 控制缩放量。
        perspective_amplitude_x: 控制x方向的透视效果。
        perspective_amplitude_y: 控制y方向的透视效果。
        patch_ratio: 控制用于创建单应性的块大小。
        max_angle: 旋转中使用的最大角度。
        allow_artifacts: 布尔值，启用应用单应性时的伪影。
        translation_overflow: 平移引起的边界伪影量。

    返回:
        形状为[8]的张量，对应于展平的单应性变换。
    """
    # 输出图像的角点
    margin = (1 - patch_ratio) / 2
    pts1 = margin + torch.tensor([[0, 0], [0, patch_ratio],
                                 [patch_ratio, patch_ratio], [patch_ratio, 0]],
                                dtype=torch.float32)
    # 输入块的角点
    pts2 = pts1.clone()

    # 随机透视和仿射扰动
    if perspective:
        if not allow_artifacts:
            perspective_amplitude_x = min(perspective_amplitude_x, margin)
            perspective_amplitude_y = min(perspective_amplitude_y, margin)
        perspective_displacement = torch.normal(0., perspective_amplitude_y/2, size=(1,))
        h_displacement_left = torch.normal(0., perspective_amplitude_x/2, size=(1,))
        h_displacement_right = torch.normal(0., perspective_amplitude_x/2, size=(1,))
        pts2 += torch.stack([
            torch.cat([h_displacement_left, perspective_displacement]),
            torch.cat([h_displacement_left, -perspective_displacement]),
            torch.cat([h_displacement_right, perspective_displacement]),
            torch.cat([h_displacement_right, -perspective_displacement])
        ])

    # 随机缩放
    if scaling:
        scales = torch.cat([torch.ones(1),
                          torch.normal(1, scaling_amplitude/2, size=(n_scales,))])
        center = torch.mean(pts2, dim=0, keepdim=True)
        scaled = (pts2 - center).unsqueeze(0) * scales.view(-1, 1, 1) + center
        if allow_artifacts:
            valid = torch.arange(1, n_scales + 1)  # 除scale=1外所有尺度都有效
        else:
            valid = torch.where(torch.all((scaled >= 0.) & (scaled <= 1.), dim=1).all(dim=1))[0]
            if len(valid) == 0:  # 如果没有有效的缩放，使用原始尺度
                valid = torch.tensor([0])
        idx = valid[torch.randint(len(valid), (1,))]
        pts2 = scaled[idx].squeeze(0)

    # 随机平移
    if translation:
        t_min = torch.min(pts2, dim=0)[0]
        t_max = torch.min(1 - pts2, dim=0)[0]
        if allow_artifacts:
            t_min += translation_overflow
            t_max += translation_overflow
        pts2 += torch.stack([
            torch.rand(1) * (t_max[0] - t_min[0]) + t_min[0],
            torch.rand(1) * (t_max[1] - t_min[1]) + t_min[1]
        ]).view(1, 2)

    # 随机旋转
    if rotation:
        angles = torch.linspace(-max_angle, max_angle, n_angles)
        angles = torch.cat([torch.zeros(1), angles])  # 以防没有有效旋转
        center = torch.mean(pts2, dim=0, keepdim=True)
        rot_mat = torch.stack([
            torch.cos(angles), -torch.sin(angles),
            torch.sin(angles), torch.cos(angles)
        ], dim=1).view(-1, 2, 2)
        rotated = torch.matmul(
            (pts2 - center).unsqueeze(0).expand(n_angles+1, -1, -1),
            rot_mat
        ) + center
        if allow_artifacts:
            valid = torch.arange(1, n_angles + 1)  # 除angle=0外所有角度都有效
        else:
            valid = torch.where(torch.all((rotated >= 0.) & (rotated <= 1.), dim=1).all(dim=1))[0]
            if len(valid) == 0:  # 如果没有有效的旋转，使用原始角度
                valid = torch.tensor([0])
        idx = valid[torch.randint(len(valid), (1,))]
        pts2 = rotated[idx].squeeze(0)

    # 缩放到实际大小
    shape = torch.tensor([shape[1], shape[0]], dtype=torch.float32)  # 不同的约定[y, x]
    pts1 = pts1 * shape.view(1, 2)
    pts2 = pts2 * shape.view(1, 2)

    def ax(p, q): return torch.tensor([p[0], p[1], 1, 0, 0, 0, -p[0] * q[0], -p[1] * q[0]], dtype=torch.float32)
    def ay(p, q): return torch.tensor([0, 0, 0, p[0], p[1], 1, -p[0] * q[1], -p[1] * q[1]], dtype=torch.float32)

    a_mat = torch.stack([f(pts1[i], pts2[i]) for i in range(4) for f in (ax, ay)])
    p_mat = torch.tensor([[pts2[i][j] for i in range(4) for j in range(2)]], dtype=torch.float32).t()
    
    # 使用PyTorch的最小二乘求解器
    homography = torch.linalg.lstsq(a_mat, p_mat).solution
    return homography.squeeze(1)

# 示例调用
shape = (256, 256)  # 图像大小为 256x256
homography = sample_homography(shape)
print(homography)

tensor([ 2.0267e+00, -2.7451e-07,  5.1764e+01,  6.8754e-01,  1.4452e+00,
        -3.4297e+00,  3.7870e-03, -6.8356e-10])
